In [0]:
------------------------------ TABLE FORMATION ------------------------------
drop table if exists call_start_logs;
drop table if exists call_end_logs;

create table call_start_logs
(
phone_number varchar(10),
start_time timestamp
);
insert into call_start_logs values
('PN1','2022-01-01 10:20:00'),('PN1','2022-01-01 16:25:00'),('PN2','2022-01-01 12:30:00')
,('PN3','2022-01-02 10:00:00'),('PN3','2022-01-02 12:30:00'),('PN3','2022-01-03 09:20:00');

create table call_end_logs
(
phone_number varchar(10),
end_time timestamp
);
insert into call_end_logs values
('PN1','2022-01-01 10:45:00'),('PN1','2022-01-01 17:05:00'),('PN2','2022-01-01 12:55:00')
,('PN3','2022-01-02 10:20:00'),('PN3','2022-01-02 12:50:00'),('PN3','2022-01-03 09:40:00');

------------------------------------------------------------------------------------------

-- Q1: /* write a query to get start time and end time of each call from below 2 tables. Also create a column of call duration in minutes. Please do take into account that there will be multiple calls from one phone number and each entry in start table has a corresponding entry in end table. */

-- Ans:
-- Method 1: Using Join and CTE
with start_logs as (
select *, row_number() over(partition by call_start_logs.phone_number order by start_time) as rn 
from call_start_logs
),
end_logs as (
select *, row_number() over(partition by call_end_logs.phone_number order by end_time) as rn 
from call_end_logs
)
select start_logs.phone_number, start_logs.start_time, end_logs.end_time,   (
    unix_timestamp(end_logs.end_time) - unix_timestamp(start_logs.start_time)
  ) / 60 as call_duration_minutes
from start_logs
join end_logs
on start_logs.phone_number = end_logs.phone_number and start_logs.rn = end_logs.rn;


-- Method 2: Using Join and Sub Query:
select A. phone_number, A.rn, A.start_time, B.end_time, (unix_timestamp(B.end_time) - unix_timestamp(A.start_time)) / 60 as call_duration_minutes
from
(select *, row_number() over (partition by phone_number order by start_time) as rn from call_start_logs) as A
inner join
(select*, row_number() over (partition by phone_number order by end_time) as rn from call_end_logs) as B
on A.phone_number = B.phone_number and A.rn = B.rn;


-- Method 2: Using Join and Sub Query:
select phone_number, rn, min(call_time) as start_time, max(call_time) as end_time, (unix_timestamp(max(call_time)) - unix_timestamp(min(call_time))) / 60 as call_duration_minutes
from
(
select phone_number, start_time as call_time, row_number() over (partition by phone_number order by start_time) as rn from call_start_logs
union
select *, row_number() over (partition by phone_number order by end_time) as rn from call_end_logs
)
group by phone_number, rn
